# Character Level LSTM in PyTorch

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
# open text file and read in data as `text`
with open('data/data.txt', 'r') as f:
    text = f.read()

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

# Tokenization

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [6]:
encoded[:100]

array([ 0, 23, 71, 65, 28, 76, 57, 24, 13,  3,  3,  3, 66, 71, 65, 65, 49,
       24, 38, 71, 64, 70, 44, 70, 76,  6, 24, 71, 57, 76, 24, 71, 44, 44,
       24, 71, 44, 70, 68, 76, 55, 24, 76, 15, 76, 57, 49, 24, 78, 14, 23,
       71, 65, 65, 49, 24, 38, 71, 64, 70, 44, 49, 24, 70,  6, 24, 78, 14,
       23, 71, 65, 65, 49, 24, 70, 14, 24, 70, 28,  6, 24, 47, 17, 14,  3,
       17, 71, 49, 16,  3,  3, 75, 15, 76, 57, 49, 28, 23, 70, 14])

# Pre-processing the data

In [7]:
# as LSTM takes the input as one hot
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [10]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [11]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])


x
 [[ 0 23 71 65 28 76 57 24 13  3]
 [ 6 47 14 24 28 23 71 28 24 71]
 [76 14 77 24 47 57 24 71 24 38]
 [ 6 24 28 23 76 24 62 23 70 76]
 [24  6 71 17 24 23 76 57 24 28]
 [62 78  6  6 70 47 14 24 71 14]
 [24 21 14 14 71 24 23 71 77 24]
 [63 54 44 47 14  6 68 49 16 24]]

y
 [[23 71 65 28 76 57 24 13  3  3]
 [47 14 24 28 23 71 28 24 71 28]
 [14 77 24 47 57 24 71 24 38 47]
 [24 28 23 76 24 62 23 70 76 38]
 [ 6 71 17 24 23 76 57 24 28 76]
 [78  6  6 70 47 14 24 71 14 77]
 [21 14 14 71 24 23 71 77 24  6]
 [54 44 47 14  6 68 49 16 24 11]]


# Defining the network with PyTorch

In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden


In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [16]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [17]:
batch_size = 128
seq_length = 100
n_epochs = 5 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs , batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 3.2467... Val Loss: 3.1755
Epoch: 1/5... Step: 20... Loss: 3.1358... Val Loss: 3.1310
Epoch: 1/5... Step: 30... Loss: 3.1427... Val Loss: 3.1214
Epoch: 1/5... Step: 40... Loss: 3.1103... Val Loss: 3.1192
Epoch: 1/5... Step: 50... Loss: 3.1402... Val Loss: 3.1173
Epoch: 1/5... Step: 60... Loss: 3.1153... Val Loss: 3.1141
Epoch: 1/5... Step: 70... Loss: 3.1043... Val Loss: 3.1110
Epoch: 1/5... Step: 80... Loss: 3.1119... Val Loss: 3.1019
Epoch: 1/5... Step: 90... Loss: 3.0955... Val Loss: 3.0791
Epoch: 1/5... Step: 100... Loss: 3.0451... Val Loss: 3.0258
Epoch: 1/5... Step: 110... Loss: 2.9552... Val Loss: 2.9273
Epoch: 1/5... Step: 120... Loss: 2.8439... Val Loss: 2.8299
Epoch: 1/5... Step: 130... Loss: 2.7381... Val Loss: 2.6921
Epoch: 2/5... Step: 140... Loss: 2.6473... Val Loss: 2.6085
Epoch: 2/5... Step: 150... Loss: 2.5784... Val Loss: 2.5399
Epoch: 2/5... Step: 160... Loss: 2.5260... Val Loss: 2.4858
Epoch: 2/5... Step: 170... Loss: 2.4568... Val Lo

# Checkpoint

In [18]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

# Predictions and top k samples

In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [20]:
#generating text
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)


In [21]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annat, as the possing, and his sack of the contoration to the fithing wher a conterarest to a gried to havone and the sead in the ferert whaters thrute had betore to sitther, and heard to the some offer the for think trat to sinces with the conderted on his hound, and she thought he saw her and when the chains stook to her, and how way hadibe it heared that still no
houre had been thought on his firth of too walking that the most still
stried of him.

"In store that's not," she said.

That was and alres and have to seill have to see humbands the charted that the stook as so the same. She were betingering of her, what at her herself with his would be have the for and the poon of a showe one, that so consterstant with the chander that that she she thought all
her trention and hersalf, taken that he had
been do that at her shunt her. He cauned her hand, and to been haddens and shided happeranges would to thow he sont has stall that to
the
mother, and the counded him. "I cond to she had be

# Loading the Checkpoint

In [22]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [23]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to all him. "I've take the mome, and I delight to
see in that her all sees, I dad't contenst. Her's state to to an if the mush to the sartice and
seminise."

And the strunge, and to saeme the conters of the prince was stild by her, and so dis not talkilg, this, and he said of his heant, whene he say to still an in one the same
and stoption, and the sone waits the saigh that had to harded the same and with the pissact, the position of his hapsed what though there had the pincess was so doon. The
sacter as hume him on the carest.

"Went, is we hor's she to be the prain of it as is should, as he had not a since. He had that imered to me on the seal and thin in some and me one of this
ary of improped. It that it and who was at it's but a shall to to som him. And to his
samp as she was ston to me to her some that
the carse at and the poolly of the
pasting on. The course, and he said.

Stepan Arkadyevitch sout his bet in ster of the mase this hould be ond
sinchion that he care

In [24]:
print(sample(loaded, 2500, top_k=5, prime="Levin asked"))

Levin asked of her suppents of his farse, who had to her showed as, thought to said in it
a pecticulter some a prothing. The farter than taking of in she who would be and he hid brongirg the parss of thore at the hapter and there was a stringe of herserf, he come to the porinice that. And he was say, and the monting of would not the pose, they, and thene were sharith them. Stry had not chere the sace, whone so her south, and all the
ridel from and to to sind his wean of the condiculed to his fartere.

So had stopped and this. He thise has begain the praith which that she showed home take to his conterstation whice she was not a posering, but the cancial, this hussed.

Annt sat is the
sear, and and she was the begine and which he was had been
at
the bighers to the
chair of the stift and the seemis at him a dreash, worls over to
the sumbreas ought his head, and stow in hom to a chinge out, which the carting was at offor to the frighed would be such on the sear, the
froncersing with the s